# 2 Recommender Systems

In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
# from utils import *

## 2.1 Movie ratings dataset

In [2]:
data = loadmat('../data/ex8_movies.mat')
Y = data['Y'] * -1 * -1 # uint8 -> int16
R = data['R'] * -1 * -1 # uint8 -> int16

print('Toy Story average rating: %.1f' % Y[0,R[0,:]].mean())

Toy Story average rating: 4.5


## 2.2 Collaborative filtering learning algorithm

In [3]:
def roll(x, y):
    """puts two matricies of any shapes to a singel vector"""
    return np.append(x, y)

def unroll(vec, shape1, shape2):
    """turns a singel vector to two matricies of shapes: shape1 and shape2"""
    rows, cols = shape1
    x = np.reshape(vec[:(rows*cols)], shape1)
    y = np.reshape(vec[(rows*cols):], shape2)
    return x, y

In [4]:
def compute_cost(parameters, y, r, shape1, shape2, l=0):
    """computes the cost"""
    x, theta = unroll(parameters, shape1, shape2)
    
    # compute perdiction cost
    pred_cost = (1/2) * (((x.dot(theta.T) - y) * r)**2).sum()
    
    # compute regulariztion cost
    reg_cost = (l/2) * (parameters**2).sum()
    
    return pred_cost + reg_cost

In [5]:
def compute_grad(parameters, y, r, shape1, shape2, l=0):
    """computes gradient of parameters"""
    x, theta = unroll(parameters, shape1, shape2)
    
    # compute prediction gradient
    x_grad = ((x.dot(theta.T) - y) * r).dot(theta)
    theta_grad = ((x.dot(theta.T) - y) * r).T.dot(x)
    pred_grad = roll(x_grad, theta_grad)
    
    # compute regulariztion gradient
    reg_grad = l * parameters
    
    return pred_grad + reg_grad

In [6]:
def check_grad(parameters, y, r, shape1, shape2, l=0):
    """check analytic grad with numeric grad"""
    
#     TODO: check why when eps is larger the numeric gradient is more acurate. 
    eps = 1e-4
    analytic_grad = compute_grad(parameters, y, r, shape1, shape2, l=l)
    
    num_examples = 50
    loc_vec = np.random.randint(0, len(parameters), num_examples)
    mean_error = 0.0
    
    for loc in loc_vec:
    
        param_p = np.copy(parameters)
        param_m = np.copy(parameters)

        param_p[loc] += eps
        param_m[loc] -= eps

        cost_p = compute_cost(param_p, y, r, shape1, shape2,l=l)
        cost_m = compute_cost(param_m, y, r, shape1, shape2,l=l)

        numeric_grad = (cost_p - cost_m)/(2*eps)    

        diff = np.abs(numeric_grad - analytic_grad[loc])
        mean_error += diff/num_examples
    
    print(mean_error)

In [7]:
def train(init_parameters, y, r, shape1, shape2, l=0):
    op_result = minimize(compute_cost, init_parameters, (y, r, shape1, shape2, l),
                         jac=compute_grad, method='L-BFGS-B')
    if not op_result.success:
        print('something whent worng!!!')
        print(op_result.message)
        return None
    
    return op_result.x




In [8]:
def get_movie_names():
    """return a list of movie names"""
    fp = open('../data/movie_ids.txt')
    content = fp.readlines()
    content = [x.strip().split(" ", 1)[-1] for x in content]
    fp.close()
    return content

## 2.3 Learning movie recommendations

In [86]:
movie_names = get_movie_names()

my_ratings = np.zeros(len(movie_names))

# rate the folwing movies:
my_ratings[0] = 1 # Toy Story (1995)
my_ratings[97] = 5 # Twelve Monkeys (1995)
my_ratings[6] = 5 # Usual Suspects, The (1995)
my_ratings[11] = 5 # Outbreak (1995)
my_ratings[53] = 1 # Shawshank Redemption, The (1994)
my_ratings[63]= 1 # While You Were Sleeping (1995)
my_ratings[65]= 1 # Forrest Gump (1994)
my_ratings[68] = 5 # Silence of the Lambs, The (1991)
my_ratings[182] = 5 # Alien (1979)
my_ratings[225] = 5 # Die Hard 2 (1990)
my_ratings[354] = 5 # Sphere (1998)
my_ratings[20] = 1 # Muppet Treasure Island (1996)

print('New user ratings:')
for i in range(len(movie_names)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], movie_names[i]))

New user ratings:
Rated 1 for Toy Story (1995)
Rated 5 for Twelve Monkeys (1995)
Rated 5 for Usual Suspects, The (1995)
Rated 1 for Muppet Treasure Island (1996)
Rated 1 for Outbreak (1995)
Rated 1 for Shawshank Redemption, The (1994)
Rated 1 for While You Were Sleeping (1995)
Rated 5 for Forrest Gump (1994)
Rated 5 for Silence of the Lambs, The (1991)
Rated 5 for Alien (1979)
Rated 5 for Die Hard 2 (1990)
Rated 5 for Sphere (1998)


In [87]:
# add a new user
newY = np.c_[Y, my_ratings]
newR = np.c_[R, (my_ratings>0)*1]

n_m, n_u = newY.shape # number of movies, number of users
n_f = 10 # number of features

# mean rating of each movie
meanY = np.zeros(n_m)
for i in range(n_m):
    meanY[i] = newY[i,newR[i,:]].mean()
# normelize data
newY = (newY.T - meanY).T

# train model
l=10
init_parameters = np.random.rand(n_m*n_f + n_u*n_f)
parameters = train(init_parameters, newY, newR, (n_m, n_f), (n_u,n_f), l=l)

X, Theta = unroll(parameters, (n_m, n_f), (n_u,n_f))

In [88]:
my_recommendetions = X.dot(Theta.T)[:,-1] + meanY
sort_idxs = np.argsort(my_recommendetions)[::-1]

num_top = 10
print('%d Top recommendations for you:' % num_top)
for i in sort_idxs[:num_top]:
    print('Predicting rating %.1f for movie %s' % (my_recommendetions[i], movie_names[i]))


10 Top recommendations for you:
Predicting rating 4.9 for movie Maya Lin: A Strong Clear Vision (1994)
Predicting rating 4.8 for movie Godfather, The (1972)
Predicting rating 4.6 for movie Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
Predicting rating 4.6 for movie Fargo (1996)
Predicting rating 4.5 for movie Wallace & Gromit: The Best of Aardman Animation (1996)
Predicting rating 4.4 for movie Wrong Trousers, The (1993)
Predicting rating 4.4 for movie Star Wars (1977)
Predicting rating 4.4 for movie Manon of the Spring (Manon des sources) (1986)
Predicting rating 4.4 for movie Haunted World of Edward D. Wood Jr., The (1995)
Predicting rating 4.4 for movie Priest (1994)
